# Poisoning of CICIDS-2017 dataset and detecting using LSD

Load in preprocessed and poisoned data to test LSD.

## Setup and import required modules

In [ ]:
!pip3 install scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from __future__ import print_function
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import csr_matrix, vstack, hstack
from scipy.sparse import coo_matrix
from keras.preprocessing.text import one_hot
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
#from sklearn.semi_supervised import label_propagation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
import math
#import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation , Flatten
from sklearn.metrics import log_loss
# from keras.optimizers import SGD
# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import UpSampling2D
# from keras.layers.convolutional import Conv2D, MaxPooling2D, MaxPooling1D
# from keras.layers.embeddings import Embedding
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D
from tensorflow.keras.layers import Embedding
from scipy import sparse
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay

#import random
import sklearn
from sklearn.metrics.pairwise import manhattan_distances
from keras.models import Model
from keras.layers import  Conv1D, multiply, GlobalMaxPool1D, Input , Lambda
import time
import argparse
#import math
from numpy import *
import os.path as osp
import scipy.sparse as sp
import pickle
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import precision_recall_curve
def plot_conf_matrix(Y_val, Y_val_preds):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(confusion_matrix(Y_val, Y_val_preds),
                     annot=True,
                     cbar=False,
                     fmt='d')
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

In [ ]:
def plot_conf_matrix_from_matrix(cm, title):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(cm,
                     annot=True,
                     cbar=False,
                     fmt='d')
    plt.title(label=title)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

Load Data

In [ ]:
import pandas as pd
# input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/converted_processed_1_percent_sample.csv', index_col='Unnamed: 0')
# input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/converted_processed_5_percent_sample.csv', index_col='Unnamed: 0')
input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/Converted-Processed-CICIDS.csv', index_col='Unnamed: 0')

input_tables = input_data.copy()
input_tables.reset_index(inplace=True)
input_tables.drop(['index'], axis=1, inplace=True)

In [ ]:
dl2 = input_tables.copy(deep=True)
print("Pre-conversion values:\n", dl2['Label'].value_counts())
for index, row in dl2.iterrows():
  if row['Label'] == 'BENIGN':
    dl2.at[index, 'Label'] = 0
  else:
    dl2.at[index, 'Label'] = 1
print("")
print("Converted values:\n", dl2['Label'].value_counts())
dl2 = dl2['Label']
dl2 = dl2.astype('int')

## Main Program Start

In [ ]:
###/content/drive/MyDrive/Dissertation/KNN
### Loading
x_train_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train.sav'
X_train = pickle.load(open(x_train_filename, 'rb'))
x_test_filename = '/content/drive/MyDrive/Dissertation/KNN/X_test.sav'
X_test = pickle.load(open(x_test_filename, 'rb'))
x_val_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val.sav'
X_val = pickle.load(open(x_val_filename, 'rb'))
y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train.sav'
Y_train = pickle.load(open(y_train_filename, 'rb'))
y_test_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_test.sav'
Y_test = pickle.load(open(y_test_filename, 'rb'))
y_val_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val.sav'
Y_val = pickle.load(open(y_val_filename, 'rb'))

In [ ]:
### Loading
model_main_filename = '/content/drive/MyDrive/Dissertation/KNN/model_main.sav'
model_main = pickle.load(open(model_main_filename, 'rb'))
X_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_NoAttack.sav'
X_train_NoAttack = pickle.load(open(X_train_NoAttack_filename, 'rb'))
Y_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_NoAttack.sav'
Y_train_NoAttack = pickle.load(open(Y_train_NoAttack_filename, 'rb'))
X_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val_NoAttack.sav'
X_val_NoAttack = pickle.load(open(X_val_NoAttack_filename, 'rb'))
Y_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val_NoAttack.sav'
Y_val_NoAttack = pickle.load(open(Y_val_NoAttack_filename, 'rb'))
Y_CNN_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_CNN_NoAttack.sav'
Y_CNN_NoAttack = pickle.load(open(Y_CNN_NoAttack_filename, 'rb'))
Y_predict_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_predict_NoAttack.sav'
Y_predict_NoAttack = pickle.load(open(Y_predict_NoAttack_filename, 'rb'))

Result of Model without attack on X_test

In [ ]:
print("********************************Result of Model without attack******************************************************************")
# loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)
#loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)
loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)

print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

#loss, accuracy = model_main.evaluate(X_val_NoAttack, Y_val_NoAttack, verbose=2)
loss, accuracy = model_main.evaluate(X_val_NoAttack, Y_val_NoAttack, verbose=2)
print('Accuracy for Validation set: %f' % (accuracy*100))
print('Loss for Train Validation set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set:: %f' % (loss))
print("                                                                   ")

con_matrix = confusion_matrix(Y_test, Y_predict_NoAttack)
print(con_matrix, "\n")

TN_NoAttack, FP_NoAttack, FN_NoAttack, TP_NoAttack = confusion_matrix((Y_test.astype(int).tolist()), Y_predict_NoAttack).ravel()
print("TN_NoAttack:",TN_NoAttack)
print("FP_NoAttack:",FP_NoAttack)
print("FN_NoAttack:",FN_NoAttack)
print("TP_NoAttack:",TP_NoAttack)
print("                                                                   ")

if (FP_NoAttack + TN_NoAttack) > 0:
        FPR_NoAttack = FP_NoAttack/(FP_NoAttack + TN_NoAttack)
        print("The FPR_NoAttack result:", FPR_NoAttack)
            
if (FP_NoAttack + TN_NoAttack) > 0:
        TPR_NoAttack=TP_NoAttack/(TP_NoAttack+FN_NoAttack)
        print("The TPR_NoAttack result=", TPR_NoAttack)
            
if (TN_NoAttack + FP_NoAttack) > 0:
    TNR_NoAttack = TN_NoAttack/(TN_NoAttack + FP_NoAttack)
    print("The TNR_NoAttack result:", TNR_NoAttack)
            
if (FN_NoAttack + TP_NoAttack) > 0:
    FNR_NoAttack = FN_NoAttack/(FN_NoAttack + TP_NoAttack)
    print("The FNR_NoAttack result:", FNR_NoAttack)
            
if ((TN_NoAttack/(TN_NoAttack + FP_NoAttack)) + (TP_NoAttack/(TP_NoAttack + FP_NoAttack))) > 0:
    AUC_NoAttack = 1/(2*((TN_NoAttack/(TN_NoAttack + FP_NoAttack)) + (TP_NoAttack/(TP_NoAttack + FP_NoAttack))))
    print("The AUC_NoAttack result:", AUC_NoAttack)
            
if  (TP_NoAttack + TN_NoAttack + FP_NoAttack + FN_NoAttack) > 0:
    ACC_NoAttack = (TP_NoAttack + TN_NoAttack)/(TP_NoAttack + TN_NoAttack + FP_NoAttack + FN_NoAttack)
    print("The ACC_NoAttack result:", ACC_NoAttack)
            
if  ((TP_NoAttack + FP_NoAttack) * (TP_NoAttack + FN_NoAttack) * (TN_NoAttack + FP_NoAttack) * (TN_NoAttack + FN_NoAttack)) > 0:
    MCC_NoAttack = (TP_NoAttack * TN_NoAttack - FP_NoAttack * FN_NoAttack)/math.sqrt((TP_NoAttack + FP_NoAttack) * (TP_NoAttack + FN_NoAttack) * (TN_NoAttack + FP_NoAttack) * (TN_NoAttack + FN_NoAttack))
    print("The Matthews correlation coefficient result:", MCC_NoAttack)
print("                                                                                                                               ")
print("*****************************************************End of Without Attack part************************************************")

## Label Flipping Attack

In [ ]:
print("*****************************************************Label Flipping Attack*****************************************************")
print("                                                                                                                               ")
#************************** 
# Finding Malware of Train data
malware_train = X_train.copy(deep=True)
y_malware_train = Y_train.copy(deep=True)
print("malware_train:", malware_train.shape)
cl_malware = list()
z_m = 0
count_m = 0
drop_list = []
for i, j in enumerate(Y_train):
#     if j == 1:
    if j == 0:
        drop_list.append(X_train.index[i])
        count_m = count_m + 1
    else:
        cl_malware.insert(z_m, 1)
        z_m = z_m + 1 
drop_list.sort(reverse=True)
malware_train.drop(drop_list, inplace=True)
y_malware_train.drop(drop_list, inplace=True)

#***************************
#Finding Benign of Train data
cl_X_train = list(Y_train) 
# benign_train=sparse.lil_matrix(X_train)
benign_train = X_train.copy(deep=True)
y_benign_train = Y_train.copy(deep=True)
print("benign_train:", benign_train.shape)
z_b = 0
count_b = 0
cl_benign = list()
drop_list = []
for i, j in enumerate(cl_X_train):
#     if j == 0:
    if j == 1:
        drop_list.append(X_train.index[i])
        count_b = count_b+1
    else:
        cl_benign.insert(z_b, 1)
        z_b = z_b+1
drop_list.sort(reverse=True)
benign_train.drop(drop_list, inplace=True)
y_benign_train.drop(drop_list, inplace=True)

print("***********Size of Each Data Part:**********")        
print("malware_train:", malware_train.shape)
print("benign_train:", benign_train.shape)
#***************************************************
row_malware_train, column_malware_train = malware_train.shape

X_train_LFA = X_train.copy(deep=True)
Y_train_LFA = Y_train.copy(deep=True)

row_train_LFA, column_train_LFA = X_train_LFA.shape

In [ ]:
### Saving
malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/malware_train.sav'
pickle.dump(malware_train, open(malware_train_filename, 'wb'))
y_malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_malware_train.sav'
pickle.dump(y_malware_train, open(y_malware_train_filename, 'wb'))
benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/benign_train.sav'
pickle.dump(benign_train, open(benign_train_filename, 'wb'))
y_benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_benign_train.sav'
pickle.dump(y_benign_train, open(y_benign_train_filename, 'wb'))
X_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_LFA.sav'
pickle.dump(X_train_LFA, open(X_train_LFA_filename, 'wb'))
Y_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_LFA.sav'
pickle.dump(Y_train_LFA, open(Y_train_LFA_filename, 'wb'))

In [ ]:
# ### Loading
# malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/malware_train.sav'
# malware_train = pickle.load(open(malware_train_filename, 'rb'))
# y_malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_malware_train.sav'
# y_malware_train = pickle.load(open(y_malware_train_filename, 'rb'))
# benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/benign_train.sav'
# benign_train = pickle.load(open(benign_train_filename, 'rb'))
# y_benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_benign_train.sav'
# y_benign_train = pickle.load(open(y_benign_train_filename, 'rb'))
# X_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_LFA.sav'
# X_train_LFA = pickle.load(open(X_train_LFA_filename, 'rb'))
# Y_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_LFA.sav'
# Y_train_LFA = pickle.load(open(Y_train_LFA_filename, 'rb'))

# row_train_LFA, column_train_LFA = X_train_LFA.shape

In [ ]:
clusterer = KMeans(n_clusters=2, random_state=10)
#X=X_train_LFA.toarray()
X = X_train_LFA.copy(deep=True)
t0 = time.time()
cluster_labels = clusterer.fit_predict(X)
sample_silhouette_values = silhouette_samples(X, cluster_labels)
#print("sample_silhouette_values = ",sample_silhouette_values)
t1 = time.time()
print("Time for creating silhouette samples: ",t1-t0)

In [ ]:
### Saving silhouette_samples
silhouette_samples_filename = '/content/drive/MyDrive/Dissertation/KNN/silhouette_samples.pkl'
pickle.dump(sample_silhouette_values, open(silhouette_samples_filename, 'wb'))
clusterer_filename = '/content/drive/MyDrive/Dissertation/KNN/clusterer.pkl'
pickle.dump(clusterer, open(clusterer_filename, 'wb'))
cluster_labels_filename = '/content/drive/MyDrive/Dissertation/KNN/cluster_labels.pkl'
pickle.dump(cluster_labels, open(cluster_labels_filename, 'wb'))

In [ ]:
# ## Loading silhouette_samples
# silhouette_samples_filename = '/content/drive/MyDrive/Dissertation/KNN/silhouette_samples.pkl'
# sample_silhouette_values = pickle.load(open(silhouette_samples_filename, 'rb'))
# clusterer_filename = '/content/drive/MyDrive/Dissertation/KNN/clusterer.pkl'
# clusterer = pickle.load(open(clusterer_filename, 'rb'))
# cluster_labels_filename = '/content/drive/MyDrive/Dissertation/KNN/cluster_labels.pkl'
# cluster_labels = pickle.load(open(cluster_labels_filename, 'rb'))

In [ ]:
t0 = time.time()
print("Values:\n", Y_train_LFA.value_counts())
flipped_Y_train = list(Y_train_LFA)
counter = 0

### For CICIDS 2017 - replace silhouette_level with these values for different levels of poisoning
### 0.05 = Original label flipping level
### 0.15 = 5% label flipping
### 0.29 = 10% label flipping
### 0.335 = 15% label flipping
### 0.36 = 20% label flipping
### 0.405 = 25% label flipping

silhouette_level = 0.405
for new_index in range(row_train_LFA): 
    if (sample_silhouette_values[new_index]<silhouette_level):
        flipped_Y_train[new_index] = abs(flipped_Y_train[new_index]-1)     # flipped_Y_train[new_index]=1
        counter = counter + 1
print("Benign data:", str(flipped_Y_train.count(0)))
print("Malicious data:", str(flipped_Y_train.count(1)))
print("Flipped counter:", counter)

t1 = time.time()
print("Time for Label Flipping Attack: ",t1-t0)
print("                                                         ") 

In [ ]:
### Saving
flipped_Y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/15/flipped_Y_train.pkl'
pickle.dump(flipped_Y_train, open(flipped_Y_train_filename, 'wb'))

In [ ]:
# ## Loading
# flipped_Y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/15/flipped_Y_train.pkl'
# flipped_Y_train = pickle.load(open(flipped_Y_train_filename, 'rb'))

In [ ]:
#**************************************************************************
model_main_LFA_Final = Sequential()
model_main_LFA_Final.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_LFA_Final.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_LFA_Final.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(Flatten())
model_main_LFA_Final.add(Dense(1, activation='sigmoid'))
model_main_LFA_Final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_main_LFA_Final.fit(X_train_LFA, np.array(flipped_Y_train), epochs=200, verbose=1)

In [ ]:
Y_predict_LFA=model_main_LFA_Final.predict(X_test, verbose=0)
Y_predict_LFA_Final=[0]*len(Y_predict_LFA)

for i in range(len(Y_predict_LFA)):
    if Y_predict_LFA[i]<0.5:
            Y_predict_LFA[i] = 0
    else:
            Y_predict_LFA[i] = 1

for i in range(len(Y_predict_LFA)):
    Y_predict_LFA_Final[i] = int(Y_predict_LFA[i])  

In [ ]:
### Saving
model_main_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/15/model_main_LFA_Final.pkl'
pickle.dump(model_main_LFA_Final, open(model_main_LFA_Final_filename, 'wb'))
Y_predict_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/15/Y_predict_LFA.pkl'
pickle.dump(Y_predict_LFA, open(Y_predict_LFA_filename, 'wb'))
Y_predict_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/15/Y_predict_LFA_Final.pkl'
pickle.dump(Y_predict_LFA_Final, open(Y_predict_LFA_Final_filename, 'wb'))

In [ ]:
# ### Loading
# model_main_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/15/model_main_LFA_Final.pkl'
# model_main_LFA_Final = pickle.load(open(model_main_LFA_Final_filename, 'rb'))
# Y_predict_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/15/Y_predict_LFA.pkl'
# Y_predict_LFA = pickle.load(open(Y_predict_LFA_filename, 'rb'))
# Y_predict_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/15/Y_predict_LFA_Final.pkl'
# Y_predict_LFA_Final = pickle.load(open(Y_predict_LFA_Final_filename, 'rb'))

Result of Model with LFA

In [ ]:
print("********************************Result of Model with LFA attack **************************************************************")
print("                                                                                                                              ")
loss, accuracy = model_main_LFA_Final.evaluate(X_train_LFA, np.array(flipped_Y_train), verbose=2)
print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main_LFA_Final.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set:: %f' % (loss))
print("                                                                   ")

TN_LFA, FP_LFA, FN_LFA, TP_LFA = confusion_matrix(Y_test,  Y_predict_LFA_Final).ravel()
print("TN_LFA:", TN_LFA)
print("FP_LFA:", FP_LFA)
print("FN_LFA:", FN_LFA)
print("TP_LFA:", TP_LFA)
print("                                                                   ")

if (FP_LFA+TN_LFA) > 0:
        FPR_LFA=FP_LFA/(FP_LFA+TN_LFA)
        print("The FPR_LFA result=", FPR_LFA)
            
if (FP_LFA+TN_LFA) > 0:
        TPR_LFA = TP_LFA/(TP_LFA+FN_LFA)
        print("The TPR_LFA result:", TPR_LFA)
            
if (TN_LFA+FP_LFA) > 0:
    TNR_LFA = TN_LFA/(TN_LFA+FP_LFA)
    print("The TNR_LFA result:", TNR_LFA)
            
if (FN_LFA+TP_LFA) > 0:
    FNR_LFA = FN_LFA/(FN_LFA+TP_LFA)
    print("The FNR_LFA result:", FNR_LFA)
            
if ((TN_LFA/(TN_LFA+FP_LFA))+(TP_LFA/(TP_LFA+FP_LFA))) > 0:
    AUC_LFA = 1/(2*((TN_LFA/(TN_LFA+FP_LFA))+(TP_LFA/(TP_LFA+FP_LFA))))
    print("The AUC_LFA result:", AUC_LFA)
            
if  (TP_LFA+TN_LFA+FP_LFA+FN_LFA) > 0:
    ACC_LFA = (TP_LFA+TN_LFA)/(TP_LFA+TN_LFA+FP_LFA+FN_LFA)
    print("The ACC_LFA result:", ACC_LFA)
            
if  ((TP_LFA+FP_LFA)*(TP_LFA+FN_LFA)*(TN_LFA+FP_LFA)*(TN_LFA+FN_LFA)) > 0:
    MCC_LFA = (TP_LFA*TN_LFA-FP_LFA*FN_LFA)/math.sqrt((TP_LFA+FP_LFA)*(TP_LFA+FN_LFA)*(TN_LFA+FP_LFA)*(TN_LFA+FN_LFA))
    print("The Matthews correlation coefficient result:", MCC_LFA)
print("                                                                                                                               ")
print("************************************************End of Label Flipping Attack part**********************************************")

********************************Result of Model with LFA attack **************************************************************
                                                                                                                              
53077/53077 - 104s - loss: 0.0494 - acc: 0.9833 - 104s/epoch - 2ms/step
Accuracy for Train set: 98.329765
Loss for Train set: 0.049409
                                                                   
17693/17693 - 38s - loss: 4.9381 - acc: 0.8396 - 38s/epoch - 2ms/step
Accuracy for Test set: 83.955640
Loss for Test set:: 4.938063
                                                                   
TN_LFA: 394830
FP_LFA: 59634
FN_LFA: 31201
TP_LFA: 80484
                                                                   
The FPR_LFA result= 0.1312183143219265
The TPR_LFA result: 0.7206339257733805
The TNR_LFA result: 0.8687816856780735
The FNR_LFA result: 0.2793660742266195
The AUC_LFA result: 0.34645634642604534
The ACC_LFA result: 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in long_scalars


Label Propagation and Label Spreading for Using in Label Based Semi-supervised Defense(LSD)

In [ ]:
print("*****************************************************Label Based Semi-supervised Defense(LSD)**********************************")
print("                                                                                                                               ")
#***********************Label Propagation and Label Spreading for Using in Label Based Semi-supervised Defense(LSD) *******************
X_train_LSD = X_train.copy(deep=True)
Y_train_LSD = flipped_Y_train[:]

X_val_LSD = X_val.copy()
Y_val_LSD = Y_val.copy(deep=True)

row_val_LSD,column_val_LSD = X_val_LSD.shape
row_train_LSD,column_train_LSD = X_train_LSD.shape

t4 = time.time()

labels = np.full(row_train_LSD, -1)
for i in range(row_val_LSD):
    labels[i] = Y_val_LSD.iloc[i]

X = X_train_LSD.copy(deep=True)
label_spread = LabelSpreading(kernel='knn', alpha=0.8)
label_propa = LabelPropagation(kernel='knn', gamma=20, n_neighbors=7, max_iter=1000, tol=0.001, n_jobs=None)
label_spread.fit(X, labels)
label_propa.fit(X, labels)
output_labels_spread = label_spread.transduction_
output_labels_propa = label_propa.transduction_

*****************************************************Label Based Semi-supervised Defense(LSD)**********************************
                                                                                                                               


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=30 was reached without convergence.
  category=ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning,


In [ ]:
### Saving
X_train_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/X_train_LSD.pkl'
pickle.dump(X_train_LSD, open(X_train_LSD_filename, 'wb'))
Y_train_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_train_LSD.pkl'
pickle.dump(Y_train_LSD, open(Y_train_LSD_filename, 'wb'))
X_val_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/X_val_LSD.pkl'
pickle.dump(X_val_LSD, open(X_val_LSD_filename, 'wb'))
Y_val_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_val_LSD.pkl'
pickle.dump(Y_val_LSD, open(Y_val_LSD_filename, 'wb'))
output_labels_spread_filename = '/content/drive/MyDrive/Dissertation/LSD/15/output_labels_spread.pkl'
pickle.dump(output_labels_spread, open(output_labels_spread_filename, 'wb'))
output_labels_propa_filename = '/content/drive/MyDrive/Dissertation/LSD/15/output_labels_propa.pkl'
pickle.dump(output_labels_propa, open(output_labels_propa_filename, 'wb'))

In [ ]:
# ### Loading
# X_train_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/X_train_LSD.pkl'
# X_train_LSD = pickle.load(open(X_train_LSD_filename, 'rb'))
# Y_train_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_train_LSD.pkl'
# Y_train_LSD = pickle.load(open(Y_train_LSD_filename, 'rb'))
# X_val_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/X_val_LSD.pkl'
# X_val_LSD = pickle.load(open(X_val_LSD_filename, 'rb'))
# Y_val_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_val_LSD.pkl'
# Y_val_LSD = pickle.load(open(Y_val_LSD_filename, 'rb'))
# output_labels_spread_filename = '/content/drive/MyDrive/Dissertation/LSD/15/output_labels_spread.pkl'
# output_labels_spread = pickle.load(open(output_labels_spread_filename, 'rb'))
# output_labels_propa_filename = '/content/drive/MyDrive/Dissertation/LSD/15/output_labels_propa.pkl'
# output_labels_propa = pickle.load(open(output_labels_propa_filename, 'rb'))

In [ ]:
#*******************Convolutional Neural Network for Using in Label Based Semi-supervised Defense(LSD) ******************************
CNN_model_for_LSD = Sequential()
CNN_model_for_LSD.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69,1)))
CNN_model_for_LSD.add(MaxPooling1D(pool_size=(4), strides=(2)))
CNN_model_for_LSD.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69,1)))
CNN_model_for_LSD.add(MaxPooling1D(pool_size=(4), strides=(2)))
CNN_model_for_LSD.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69,1)))
CNN_model_for_LSD.add(Flatten())
CNN_model_for_LSD.add(Dense(1, activation='sigmoid'))
CNN_model_for_LSD.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
CNN_model_for_LSD.fit(X_train_LSD, np.asarray(Y_train_LSD), epochs=200, verbose=1)

Y_predict_CNN_for_LSD = CNN_model_for_LSD.predict(X_train_LSD, verbose=0)

Y_predict_CNN_LSD_Final = [0]*len(Y_predict_CNN_for_LSD)
for i in range(len(Y_predict_CNN_for_LSD)):
    if Y_predict_CNN_for_LSD[i] < 0.5:
            Y_predict_CNN_for_LSD[i] = 0
    else:
            Y_predict_CNN_for_LSD[i] = 1

for i in range(len(Y_predict_CNN_for_LSD)):
    Y_predict_CNN_LSD_Final[i]= int(Y_predict_CNN_for_LSD[i])

In [ ]:
### Saving
CNN_model_for_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/CNN_model_for_LSD.pkl'
pickle.dump(CNN_model_for_LSD, open(CNN_model_for_LSD_filename, 'wb'))
Y_predict_CNN_for_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_CNN_for_LSD.pkl'
pickle.dump(Y_predict_CNN_for_LSD, open(Y_predict_CNN_for_LSD_filename, 'wb'))
Y_predict_CNN_LSD_Final_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_CNN_LSD_Final.pkl'
pickle.dump(Y_predict_CNN_LSD_Final, open(Y_predict_CNN_LSD_Final_filename, 'wb'))

In [ ]:
# ### Loading
# CNN_model_for_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/CNN_model_for_LSD.pkl'
# CNN_model_for_LSD = pickle.load(open(CNN_model_for_LSD_filename, 'rb'))
# Y_predict_CNN_for_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_CNN_for_LSD.pkl'
# Y_predict_CNN_for_LSD = pickle.load(open(Y_predict_CNN_for_LSD_filename, 'rb'))
# Y_predict_CNN_LSD_Final_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_CNN_LSD_Final.pkl'
# Y_predict_CNN_LSD_Final = pickle.load(open(Y_predict_CNN_LSD_Final_filename, 'rb'))

In [ ]:
#*******************************************Voting Between CNN , label Propagation and Label Spreading**************************     
# t4 = time.time() ### Uncomment for partial run when skipping earlier step
Y_predict_LSD_Final = [0]*len(Y_train)
for i in range(len(Y_train)):
    c = Y_train_LSD[i] + Y_predict_CNN_LSD_Final[i] + output_labels_propa[i] + output_labels_spread[i]
    if 2 <= c:
        Y_predict_LSD_Final[i] = 1
    else:
        Y_predict_LSD_Final[i] = 0
t5 = time.time()
print("Time for Label Based Semi-supervised Defense =",t5-t4)
print("                                                                                                                               ")


In [ ]:
### Saving
Y_predict_LSD_Final_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_LSD_Final.pkl'
pickle.dump(Y_predict_LSD_Final, open(Y_predict_LSD_Final_filename, 'wb'))

In [ ]:
# ### Loading
# Y_predict_LSD_Final_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_LSD_Final.pkl'
# Y_predict_LSD_Final = pickle.load(open(Y_predict_LSD_Final_filename, 'rb'))

In [ ]:
#*********************************************************************************************************************************
model_main_LSD = Sequential()
model_main_LSD.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LSD.add(MaxPooling1D(pool_size = (4), strides=(2)))
model_main_LSD.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LSD.add(MaxPooling1D(pool_size = (4), strides=(2)))
model_main_LSD.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LSD.add(Flatten())
model_main_LSD.add(Dense(1, activation='sigmoid'))
model_main_LSD.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_main_LSD.fit(X_train_LSD, np.asarray(Y_predict_LSD_Final), epochs=200, verbose=1)
    
Y_predict_LSD_Defense=model_main_LSD.predict(X_test, verbose=0)
Y_predict_LSD_Defense_Final=[0]*len(Y_predict_LSD_Defense)

for i in range(len(Y_predict_LSD_Defense)):
    if Y_predict_LSD_Defense[i]<0.5:
            Y_predict_LSD_Defense[i]=0
    else:
            Y_predict_LSD_Defense[i]=1

for i in range(len(Y_predict_LSD_Defense)):
    Y_predict_LSD_Defense_Final[i] = int(Y_predict_LSD_Defense[i])

In [ ]:
### Saving
model_main_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/model_main_LSD.pkl'
pickle.dump(model_main_LSD, open(model_main_LSD_filename, 'wb'))
Y_predict_LSD_Defense_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_LSD_Defense.pkl'
pickle.dump(Y_predict_LSD_Defense, open(Y_predict_LSD_Defense_filename, 'wb'))
Y_predict_LSD_Defense_Final_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_LSD_Defense_Final.pkl'
pickle.dump(Y_predict_LSD_Defense_Final, open(Y_predict_LSD_Defense_Final_filename, 'wb'))

In [ ]:
# ### Loading
# model_main_LSD_filename = '/content/drive/MyDrive/Dissertation/LSD/15/model_main_LSD.pkl'
# model_main_LSD = pickle.load(open(model_main_LSD_filename, 'rb'))
# Y_predict_LSD_Defense_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_LSD_Defense.pkl'
# Y_predict_LSD_Defense = pickle.load(open(Y_predict_LSD_Defense_filename, 'rb'))
# Y_predict_LSD_Defense_Final_filename = '/content/drive/MyDrive/Dissertation/LSD/15/Y_predict_LSD_Defense_Final.pkl'
# Y_predict_LSD_Defense_Final = pickle.load(open(Y_predict_LSD_Defense_Final_filename, 'rb'))

Result of Model after Label Based Semi-supervised Defense(LSD)

In [ ]:
#**************************************Result of Model after Label Based Semi-supervised Defense(LSD)**********************************
print("************************Result of Model after Label Based Semi-supervised Defense(LSD)*****************************************")
print("                                                                                                                               ")
loss, accuracy = model_main.evaluate(X_train, np.asarray(Y_predict_LSD_Final), verbose=2)
print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set: %f' % (loss))
print("                                                                   ")

TN_LSD, FP_LSD, FN_LSD, TP_LSD = confusion_matrix(Y_test, Y_predict_LSD_Defense_Final).ravel()
print("TN_LSD:",TN_LSD)
print("FP_LSD:",FP_LSD)
print("FN_LSD:",FN_LSD)
print("TP_LSD:",TP_LSD)
print("                                                                   ")

if (FP_LSD + TN_LSD) > 0:
        FPR_LSD = FP_LSD/(FP_LSD + TN_LSD)
        print("The FPR_LSD result:", FPR_LSD)
            
if (FP_LSD + TN_LSD) > 0:
        TPR_LSD = TP_LSD/(TP_LSD + FN_LSD)
        print("The TPR_LSD result:", TPR_LSD)
            
if (TN_LSD+FP_LSD) > 0:
    TNR_LSD = TN_LSD/(TN_LSD + FP_LSD)
    print("The TNR_LSD result:", TNR_LSD)
            
if (FN_LSD + TP_LSD) > 0:
    FNR_LSD = FN_LSD/(FN_LSD+TP_LSD)
    print("The FNR_LSD result:", FNR_LSD)
            
if ((TN_LSD/(TN_LSD+FP_LSD)) + (TP_LSD/(TP_LSD+FP_LSD))) > 0:
    AUC_LSD = 1/(2*((TN_LSD/(TN_LSD+FP_LSD)) + (TP_LSD/(TP_LSD+FP_LSD))))
    print("The AUC result:", AUC_LSD)
            
if  (TP_LSD + TN_LSD + FP_LSD + FN_LSD) > 0:   
    ACC_LSD = (TP_LSD + TN_LSD)/(TP_LSD + TN_LSD + FP_LSD + FN_LSD)
    print("The ACC result:", ACC_LSD)
            
if  ((TP_LSD + FP_LSD) * (TP_LSD + FN_LSD) * (TN_LSD + FP_LSD) * (TN_LSD + FN_LSD)) > 0:
    MCC_LSD = (TP_LSD * TN_LSD - FP_LSD*FN_LSD)/math.sqrt((TP_LSD + FP_LSD) * (TP_LSD + FN_LSD) * (TN_LSD+FP_LSD) * (TN_LSD+FN_LSD))
    print("The Matthews correlation coefficient result:", MCC_LSD) 
print("                                                                                                                               ")
print("*****************************************************End of Label Based Semi-supervised Defense(LSD)***************************")   